In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
L = 1.0          # Length of the domain
T = 0.5          # Total simulation time
Nx = 100         # Number of spatial points
Nt = 200         # Number of time steps
alpha = 0.01     # Thermal diffusivity
dx = L / (Nx - 1)  # Spatial step size
dt = T / Nt        # Time step size
x = np.linspace(0, L, Nx)
time = np.linspace(0, T, Nt)

# Initial condition (e.g., Gaussian)
u0 = np.exp(-100 * (x - 0.5)**2)

# Solve the full-order model (finite difference method)
U_full = np.zeros((Nx, Nt))  # Solution matrix
U_full[:, 0] = u0

for n in range(0, Nt - 1):
    # Apply finite difference scheme for heat equation
    U_full[1:-1, n+1] = U_full[1:-1, n] + alpha * dt / dx**2 * (
        U_full[2:, n] - 2 * U_full[1:-1, n] + U_full[:-2, n]
    )

# Perform Singular Value Decomposition (SVD) to find POD modes
U_snapshots = U_full[:, ::5]  # Take snapshots every 5 time steps
Phi, Sigma, Psi_T = np.linalg.svd(U_snapshots, full_matrices=False)

# Retain dominant POD modes
r = 5  # Number of retained modes
Phi_r = Phi[:, :r]

# Project initial condition onto the reduced space
a0 = np.dot(Phi_r.T, U_full[:, 0])

# Galerkin projection: Precompute matrices
M = np.eye(r)  # Mass matrix (identity due to orthonormality of POD modes)
K = np.zeros((r, r))  # Stiffness matrix

for i in range(r):
    for j in range(r):
        K[i, j] = alpha * np.dot(Phi_r[:, i], np.gradient(np.gradient(Phi_r[:, j], dx), dx))

# Solve the reduced system of ODEs
a = np.zeros((r, Nt))
a[:, 0] = a0

for n in range(0, Nt - 1):
    a[:, n+1] = a[:, n] + dt * (-np.dot(K, a[:, n]))

# Reconstruct solution from POD modes
U_pod = np.dot(Phi_r, a)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(x, U_full[:, -1], label="Full Model", linestyle='-')
plt.plot(x, U_pod[:, -1], label="POD-ROM", linestyle='--')
plt.xlabel('x')
plt.ylabel('u(x,t)')
plt.title('1D Heat Equation: Full Model vs POD-ROM')
plt.legend()
plt.grid(True)
plt.show()